In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import scipy.cluster.hierarchy as sch
import glob

In [2]:
l = [pd.read_csv(filename) for filename in sorted(glob.glob("output/centroids/*.csv"))]
dados = pd.concat(l, axis=0, sort=False)
dados['DATETIME'] =  pd.to_datetime(dados['DATETIME'], format='%Y-%m-%d %H:%M:%S')
dados = dados.set_index('DATETIME')
dados.index = dados.index.map(lambda x: x.replace(second=0))

freqtime = pd.date_range(start=dados.index.min(), end=dados.index.max(), freq='12T')
t1 = freqtime[0]
t2 = freqtime[1]
t3 = freqtime[2]

In [3]:
time1 = dados.loc[t1]
time2 = dados.loc[t2]
time3 = dados.loc[t3]

In [4]:
time1

,N_Cluster,ID_CLUS,LAT,LON,DIST,IND_X,IND_Y,T_RELATION,RAIN_FALL,DBz
DATETIME,,,,,,,,,,
2014-09-28 19:24:00,12,0,-4.209177,-60.496387,129.979844,61,92,SW,115.717804,56.024703
2014-09-28 19:24:00,12,1,-2.268807,-60.567352,116.431544,169,88,NW,57.591373,51.176019
2014-09-28 19:24:00,12,2,-3.561566,-58.605930,160.661355,97,197,SE,40.754768,48.773155
2014-09-28 19:24:00,12,3,-2.304838,-60.315643,99.995682,167,102,NW,61.735973,51.658913
2014-09-28 19:24:00,12,4,-2.771915,-60.441680,65.067066,141,95,NW,59.813438,51.439080
2014-09-28 19:24:00,12,5,-3.921481,-59.145630,127.075574,77,167,SE,41.205261,48.849543
2014-09-28 19:24:00,12,6,-2.052759,-61.178467,179.137221,181,54,NW,44.350128,49.360618
2014-09-28 19:24:00,12,7,-2.286910,-60.117860,96.300329,168,113,NW,35.244781,47.763817
2014-09-28 19:24:00,12,8,-3.975717,-60.244125,95.659669,74,106,SW,32.044018,47.102251


In [5]:
time2

,N_Cluster,ID_CLUS,LAT,LON,DIST,IND_X,IND_Y,T_RELATION,RAIN_FALL,DBz
DATETIME,,,,,,,,,,
2014-09-28 19:36:00,11,0,-4.173204,-60.568420,130.147843,63,88,SW,112.079689,55.802731
2014-09-28 19:36:00,11,1,-2.016774,-61.232365,186.247937,183,51,NW,85.160400,53.894103
2014-09-28 19:36:00,11,2,-3.274401,-61.161720,130.752088,113,55,SW,54.635250,50.809867
2014-09-28 19:36:00,11,3,-2.304872,-60.153824,95.011285,167,111,NW,66.665298,52.192697
2014-09-28 19:36:00,11,4,-2.789865,-60.477665,66.994555,140,93,NW,31.207119,46.918359
2014-09-28 19:36:00,11,5,-2.286735,-60.639280,119.422815,168,84,NW,54.036713,50.733323
2014-09-28 19:36:00,11,6,-3.525661,-58.623980,157.645567,99,196,SE,41.548103,48.907119
2014-09-28 19:36:00,11,7,-3.041134,-60.873592,98.713464,126,71,NW,29.676811,46.568976
2014-09-28 19:36:00,11,8,-4.065336,-59.307587,126.723716,69,158,SE,26.440586,45.766637


In [6]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist

def calcDist(t1,t2):

    t1 = t1[['ID_CLUS','IND_X','IND_Y']]
    t1_data = t1.values.tolist()

    t2 = t2[['ID_CLUS','IND_X','IND_Y']]
    t2_data = t2.values.tolist()


    t1_ = pd.DataFrame(data=t1_data, columns=['T1', 'X', 'Y'])
    t2_ = pd.DataFrame(data=t2_data, columns=['T2', 'X', 'Y'])

    mask = cdist(t2_[['X', 'Y']].values, t1_[['X', 'Y']].values,'euclidean') < 5

    arr = []

    def zone(x):
        arr.append ( t1_[x].T1.values[0] if x.any() else np.nan)

    result = t2_.drop(['X', 'Y'], axis=1)
    np.apply_along_axis(zone, True, mask)
    result['T1'] = arr
    
    return result

In [7]:
calcDist(time1,time2)

,T2,T1
0,0,0.0
1,1,6.0
2,2,NaN
3,3,7.0
4,4,4.0
5,5,1.0
6,6,2.0
7,7,NaN
8,8,NaN
9,9,NaN


In [8]:
from geopy.distance import geodesic

In [39]:
traj = pd.DataFrame(columns=['START_TIME','END_TIME','T1','T2'])

t1t2 = calcDist(time1,time2)
for i,row in t1t2.iterrows():
    a = time1.loc[time1['ID_CLUS'] == row[0]]
    b = time2.loc[time2['ID_CLUS'] == row[0]]
    
    p1 = (str(a.LAT[0])+','+str(a.LON[0]))
    p2 = (str(b.LAT[0])+','+str(b.LON[0]))

    traj = traj.append({'START_TIME':a.index[0],'END_TIME':b.index[0],
                        'T1':row[1],'T2':t1t2.T2[i]},ignore_index=True)

traj = traj.sort_values('T1')
traj = traj.dropna()
traj['T1'] = traj['T1'].astype(int)

In [40]:
traj

,START_TIME,END_TIME,T1,T2
0,2014-09-28 19:24:00,2014-09-28 19:36:00,0,0
5,2014-09-28 19:24:00,2014-09-28 19:36:00,1,5
6,2014-09-28 19:24:00,2014-09-28 19:36:00,2,6
4,2014-09-28 19:24:00,2014-09-28 19:36:00,4,4
1,2014-09-28 19:24:00,2014-09-28 19:36:00,6,1
3,2014-09-28 19:24:00,2014-09-28 19:36:00,7,3


In [48]:
track = pd.DataFrame(columns=['START_TIME','END_TIME','T1','T2','X1','Y1','X2','Y2'])

for i, row in traj.iterrows():
    ts1 = (time1.loc[time1['ID_CLUS']==row.T1])
    ts2 = (time2.loc[time2['ID_CLUS']==row.T2])
    track = track.append({'START_TIME':ts1.index[0],'END_TIME':ts2.index[0],
                          'T1':ts1.ID_CLUS[0],'T2':ts2.ID_CLUS[0],
                          'X1':ts1.IND_X[0],'Y1':ts1.IND_Y[0],
                          'X2':ts2.IND_X[0],'Y2':ts2.IND_Y[0],},ignore_index=True) 
track

,START_TIME,END_TIME,T1,T2,X1,Y1,X2,Y2
0,2014-09-28 19:24:00,2014-09-28 19:36:00,0,0,61,92,63,88
1,2014-09-28 19:24:00,2014-09-28 19:36:00,1,5,169,88,168,84
2,2014-09-28 19:24:00,2014-09-28 19:36:00,2,6,97,197,99,196
3,2014-09-28 19:24:00,2014-09-28 19:36:00,4,4,141,95,140,93
4,2014-09-28 19:24:00,2014-09-28 19:36:00,6,1,181,54,183,51
5,2014-09-28 19:24:00,2014-09-28 19:36:00,7,3,168,113,167,111


In [ ]:
tracker = pd.DataFrame()

for i, row in traj.iterrows():
    ts1 = (time1.loc[time1['ID_CLUS']==row.T1])
    ts2 = (time2.loc[time2['ID_CLUS']==row.T2])
    tracker = tracker.append(ts1)
    tracker = tracker.append(ts2)
    
tracker

In [ ]:
time1

In [ ]:
time2

In [ ]:
dataset = time1.append(time2)
dataset.head()

In [ ]:
dataset2 = time2.append(time3)
dataset2.head()

In [ ]:
X = dataset.iloc[:,[6,5]].values
X2 = dataset.iloc[:,[5,6]].values
X

In [ ]:
dendrogram = sch.dendrogram(sch.linkage(x, method='ward'))

In [ ]:
dendrogram = sch.dendrogram(sch.linkage(X2, method='ward'))

In [ ]:
model = AgglomerativeClustering(n_clusters=None, affinity='euclidean', linkage='ward',distance_threshold=10)
model.fit(X)
labels = model.labels_

In [ ]:
plt.scatter(X[labels==0, 0], X[labels==0, 1], s=50, marker='o', color='red')
plt.scatter(X[labels==1, 0], X[labels==1, 1], s=50, marker='o', color='blue')
plt.scatter(X[labels==2, 0], X[labels==2, 1], s=50, marker='o', color='green')
plt.scatter(X[labels==3, 0], X[labels==3, 1], s=50, marker='o', color='purple')
plt.scatter(X[labels==4, 0], X[labels==4, 1], s=50, marker='o', color='orange')
plt.scatter(X[labels==5, 0], X[labels==5, 1], s=50, marker='o', color='black')
# plt.scatter(X[:,0],X[:,1], s=50,marker='+',color='black')
# plt.scatter(time2['IND_X'],time2['IND_Y'],marker='+')
plt.show()

In [ ]:
plt.scatter(X[:,0],X[:,1])
plt.scatter(X[labels==0, 0], X[labels==0, 1])

In [ ]:
plt.scatter(X[labels==0, 0], X[labels==0, 1])

In [ ]:
X[labels][:,0]

In [ ]:
model = AgglomerativeClustering(n_clusters=None, affinity='euclidean', linkage='ward',distance_threshold=1)
model.fit(X2)
labels = model.labels_
X2[labels==0, 0], X2[labels==0, 1]

In [ ]:
plt.scatter(X2[labels==0, 0], X2[labels==0, 1], s=50, marker='o', color='red')
plt.scatter(X2[labels==1, 0], X2[labels==1, 1], s=50, marker='o', color='blue')
plt.scatter(X2[labels==2, 0], X2[labels==2, 1], s=50, marker='o', color='green')
plt.scatter(X2[labels==3, 0], X2[labels==3, 1], s=50, marker='o', color='purple')
plt.scatter(X2[labels==4, 0], X2[labels==4, 1], s=50, marker='o', color='orange')
plt.scatter(X2[labels==5, 0], X2[labels==5, 1], s=50, marker='o', color='black')
plt.show()

In [ ]:
import scipy

mat = scipy.spatial.distance.cdist(time1[['IND_X','IND_Y']], 
                              time2[['IND_X','IND_Y']], metric='euclidean')

In [ ]:
mat

In [ ]:
import numpy as np
from scipy import spatial

xy1 = time1.iloc[:,[6,5]].values
xy2 = time2.iloc[:,[6,5]].values


# This solution is optimal when xy2 is very large
tree = spatial.cKDTree(xy2)
mindist, minid = tree.query(xy1)
print(mindist)

# This solution by @denis is OK for small xy2
mindist = np.min(spatial.distance.cdist(xy1, xy2), axis=1)
print(mindist)

In [ ]:
xy1

In [ ]:
import numpy as np
P = np.add.outer(np.sum(xy1**2, axis=1), np.sum(xy2**2, axis=1))
N = np.dot(xy1, xy2.T)
dists = np.sqrt(P - 2*N)
dists

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
df =pd.DataFrame(euclidean_distances(time1[['IND_X','IND_Y']],time2[['IND_X','IND_Y']]))
# df.index =  time1[['IND_X','IND_Y']].index
# df.columns = time2[['IND_X','IND_Y']].index
df['min_distance'] = df.min(axis=1)
df['min_distance_id'] = df.idxmin(axis=1)

In [ ]:
df[['min_distance','min_distance_id']]

In [ ]:
time1[['IND_X','IND_Y']]

In [ ]:
time2[['IND_X','IND_Y']]

In [ ]:
import math
def calculateDistance(x1,y1,x2,y2):  
    dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)  
    return dist  

calculateDistance(169,88,168,84)  

In [ ]:
df[['min_distance','min_distance_id']]

In [ ]:
KDTree = scipy.spatial.cKDTree(time1[['IND_X','IND_Y']])
distances, indexes = KDTree.query(time2[['IND_X','IND_Y']], n_jobs=-1)

In [ ]:
import numpy as np
from scipy.spatial.distance import cdist

def feature_dist(input):
    """
    Takes a labeled array as returned by scipy.ndimage.label and 
    returns an intra-feature distance matrix.
    """
    I, J = np.nonzero(input)
    labels = input[I,J]
    coords = np.column_stack((I,J))

    sorter = np.argsort(labels)
    labels = labels[sorter]
    coords = coords[sorter]

    sq_dists = cdist(coords, coords, 'sqeuclidean')

    start_idx = np.flatnonzero(np.r_[1, np.diff(labels)])
    nonzero_vs_feat = np.minimum.reduceat(sq_dists, start_idx, axis=1)
    feat_vs_feat = np.minimum.reduceat(nonzero_vs_feat, start_idx, axis=0)

    return np.sqrt(feat_vs_feat)

feature_dist(zone_data,points_data)

In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist

# zone_data = [['A1', 704178, 2359686], ['A2', 670179, 2343883], ['A3', 723439, 2346826],
#         ['A4', 718530, 2377080], ['A5', 679772, 2379091]]

t2 = time1[['ID_CLUS','IND_X','IND_Y']]
zone_data = t1.values.tolist()

t1 = time2[['ID_CLUS','IND_X','IND_Y']]
points_data = t2.values.tolist()

# points_data = [['P1 ', 675176, 2373313], ['P2', 684905, 2378956],
#         ['P3', 675002, 2352012], ['P4', 675933, 2381910],
#         ['P5', 685268, 2364044], ['P6', 673324, 2377060],
#         ['P7', 684222, 2371631], ['P8', 701418, 2356943],
#         ['P9', 700891, 2362305], ['P10', 706972, 2358842],
#         ['P11', 706904, 2364451], ['P12', 721197, 2347368],
#         ['P13', 726825, 2345518], ['P14', 725521, 2351631],
#         ['P15', 721214, 2353052], ['P16', 700920, 2369710],
#         ['P17', 695029, 2365463], ['P18', 715987, 2376662],
#         ['P19', 721979, 2379020], ['P20', 716318, 2379221],
#         ['P21', 673892, 2345205], ['P22', 689204, 2354791],
#         ['P23', 667520, 2347603], ['P24', 673688, 2348698],
#         ['P25', 666493, 2362489], ['P26', 698172, 2350498],
#         ['P27', 720295, 2381290], ['P28', 681206, 2383585],
#         ['P29', 680696, 2377118], ['P30', 695803, 2359471]]

zones = pd.DataFrame(data=zone_data, columns=['LABEL', 'X', 'Y'])
points = pd.DataFrame(data=points_data, columns=['LABEL', 'X', 'Y'])

In [ ]:
mask = cdist(points[['X', 'Y']].values, zones[['X', 'Y']].values,'euclidean') < 5

arr = []

def zone(x):
    arr.append ( zones[x].LABEL.values[0] if x.any() else 'NaN')

#     return zones[x].LABEL if x.any() else ''


result = points.drop(['X', 'Y'], axis=1)
np.apply_along_axis(zone, True, mask)
result['zone'] = arr
# result['zone'] = np.apply_along_axis(zone, 1, mask)

In [ ]:
np.apply_along_axis(zone, True, mask)
result['zone'] = arr

In [ ]:
aux

In [ ]:
result['zone'] = arr

In [ ]:
result

In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist

zone_data = [['A1', 704178, 2359686], ['A2', 670179, 2343883], ['A3', 723439, 2346826],
        ['A4', 718530, 2377080], ['A5', 679772, 2379091]]

points_data = [['P1 ', 675176, 2373313], ['P2', 684905, 2378956],
        ['P3', 675002, 2352012], ['P4', 675933, 2381910],
        ['P5', 685268, 2364044], ['P6', 673324, 2377060],
        ['P7', 684222, 2371631], ['P8', 701418, 2356943],
        ['P9', 700891, 2362305], ['P10', 706972, 2358842],
        ['P11', 706904, 2364451], ['P12', 721197, 2347368],
        ['P13', 726825, 2345518], ['P14', 725521, 2351631],
        ['P15', 721214, 2353052], ['P16', 700920, 2369710],
        ['P17', 695029, 2365463], ['P18', 715987, 2376662],
        ['P19', 721979, 2379020], ['P20', 716318, 2379221],
        ['P21', 673892, 2345205], ['P22', 689204, 2354791],
        ['P23', 667520, 2347603], ['P24', 673688, 2348698],
        ['P25', 666493, 2362489], ['P26', 698172, 2350498],
        ['P27', 720295, 2381290], ['P28', 681206, 2383585],
        ['P29', 680696, 2377118], ['P30', 695803, 2359471]]

zones = pd.DataFrame(data=zone_data, columns=['LABEL', 'X', 'Y'])
points = pd.DataFrame(data=points_data, columns=['LABEL', 'X', 'Y'])

In [ ]:
zones = pd.DataFrame(data=zone_data, columns=['LABEL', 'X', 'Y'])
points = pd.DataFrame(data=points_data, columns=['LABEL', 'X', 'Y'])
mask = cdist(points[['X', 'Y']].values, zones[['X', 'Y']].values, 'euclidean') < 10000


def zone(x):
    return zones[x].LABEL if x.any() else ''


result = points.drop(['X', 'Y'], axis=1)
result['zone'] = np.apply_along_axis(zone, 1, mask)

In [ ]:
result